In [1]:
# Import libraries
from ngsolve import *
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

$$\kappa = 
\begin{bmatrix}
\mu_1 & \mu_2 \\
\mu_3 & \mu_4
\end{bmatrix}
$$


In [97]:
mu1 = Parameter(1.0)
mu2 = Parameter(0.0)
mu3 = Parameter(0.0)
mu4 = Parameter(1.0)

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
V = H1(mesh,order=1,dirichlet=[1,2,3,4])
# anisotropic diffusion tensor

k11 = mu1
k12 = mu2
k21 = mu3
k22 = mu4
kappa = CoefficientFunction(((k11, k12), (k21, k22)), dims=(2, 2))

u = V.TrialFunction()
v = V.TestFunction()

u_man = 16*x*(1-x)*y*(1-y) # Manufactured solution
f_man = -( (k11*u_man.Diff(x) + k12*u_man.Diff(y)).Diff(x) +
           (k21*u_man.Diff(x) + k22*u_man.Diff(y)).Diff(y) )

f = LinearForm(V)
f += f_man * v * dx

a = BilinearForm(V, symmetric=False)
a += (kappa*Grad(u))*Grad(v) * dx
a.Assemble()
f.Assemble()

gfu = GridFunction(V)
gfu.vec.data = a.mat.Inverse(V.FreeDofs(),inverse="sparsecholesky") * f.vec

from ngsolve.webgui import Draw
# Draw(gfu, mesh, "u")


In [98]:
from ngsolve import *
from netgen.geom2d import unit_square
import numpy as np

# --- Mesh and function space (enforce coercivity with Dirichlet BCs) ---
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
V = H1(mesh, order=1, dirichlet=".*")
u, v = V.TnT()

# --- Mass matrix for M-orthonormalization ---
m = BilinearForm(V)
m += u * v * dx
m.Assemble()
M = m.mat

# --- Right-hand side ---
f = LinearForm(V)
f += 1 * v * dx
f.Assemble()

# --- Helper: force SPD diffusion tensor ---
def spd_kappa(mu1, mu2, mu3, mu4, eps=1e-8):
    # symmetrize and add small diagonal to keep eigenvalues away from 0
    a11, a12, a21, a22 = mu1, mu2, mu3, mu4
    s12 = 0.5 * (a12 + a21)
    return CoefficientFunction(((a11 + eps, s12),
                                (s12, a22 + eps)), dims=(2, 2))

# --- Parameter samples (avoid singular/indefinite cases) ---
mu_samples = [
    (1.0, 0.0, 0.0, 1.0),   # identity
    (2.0, 0.5, 0.5, 1.5),   # SPD
    (0.7, 0.2, 0.2, 1.3),   # SPD
    (1.5, 0.0, 0.0, 0.8),   # SPD diagonal
    (1.0, 0.3, 0.3, 2.0),   # SPD
    (0.8, -0.2, -0.2, 1.6), # SPD (near anisotropic)
]

# --- Generate snapshots (use Cholesky only for SPD; safe now) ---
snapshots = []
for mu1, mu2, mu3, mu4 in mu_samples:
    kappa_mu = spd_kappa(mu1, mu2, mu3, mu4)
    a_mu = BilinearForm(V, symmetric=True)
    a_mu += (kappa_mu * Grad(u)) * Grad(v) * dx
    a_mu.Assemble()

    gfu_mu = GridFunction(V)
    gfu_mu.vec.data = a_mu.mat.Inverse(V.FreeDofs(), inverse="sparsecholesky") * f.vec
    vec_copy = gfu_mu.vec.CreateVector()
    vec_copy.data = gfu_mu.vec
    snapshots.append(vec_copy)


# --- M-orthonormal Gram–Schmidt ---
def gram_schmidt_M(vectors, M, tol=1e-10):
    RB = []
    for v in vectors:
        w = v.CreateVector()
        w.data = v
        for q in RB:
            # projection coefficient in M-inner product: <q, M w>
            Mw = M * w
            alpha = InnerProduct(q, Mw)
            w.data -= alpha * q
        # M-norm
        Mw = M * w
        nrm = sqrt(InnerProduct(w, Mw))
        if nrm > tol:
            w.data /= nrm
            RB.append(w)
    return RB

RB = gram_schmidt_M(snapshots, M)

# Sanity check: M-orthonormality
def check_orthonormality(RB, M):
    n = len(RB)
    G = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            G[i, j] = InnerProduct(RB[i], M * RB[j])
    offdiag = np.max(np.abs(G - np.eye(n)))
    print("Max |G - I| (M-inner product):", offdiag)

check_orthonormality(RB, M)

# --- Projection function with conditioning guard ---
def project_to_RB(mu1, mu2, mu3, mu4, cond_thresh=1e10, reg_rel=1e-10):
    kappa_mu = spd_kappa(mu1, mu2, mu3, mu4)
    a_mu = BilinearForm(V, symmetric=True)
    a_mu += (kappa_mu * Grad(u)) * Grad(v) * dx
    a_mu.Assemble()

    nrb = len(RB)
    A_rb = np.zeros((nrb, nrb))
    rhs_rb = np.zeros(nrb)

    for i in range(nrb):
        Ai = a_mu.mat * RB[i]
        for j in range(nrb):
            A_rb[j, i] = InnerProduct(RB[j], Ai)  # A_rb = U^T A U
        rhs_rb[i] = InnerProduct(RB[i], f.vec)    # b_rb = U^T f

    # Conditioning and optional Tikhonov regularization
    c = np.linalg.cond(A_rb)
    print("cond(A_rb) =", c)
    if not np.isfinite(c) or c > cond_thresh:
        tau = reg_rel * np.linalg.norm(A_rb, 2)
        A_rb = A_rb + tau * np.eye(nrb)
        print(f"Applied Tikhonov regularization tau = {tau:g}")

    # Solve reduced system robustly
    u_rb = np.linalg.solve(A_rb, rhs_rb)

    # Lift back
    gfu_rb = GridFunction(V)
    gfu_rb.vec[:] = 0
    for i in range(nrb):
        gfu_rb.vec.data += u_rb[i] * RB[i]

    return gfu_rb

# --- Test on a training parameter (should reconstruct cleanly) ---
gfu_rb = project_to_RB(1.0, 0, 0, 1.0)
print("RB vector norm:", Norm(gfu_rb.vec))
print(gfu_rb)

# from ngsolve.webgui import Draw
# w = Draw(gfu_rb, mesh, "u")
# w


Max |G - I| (M-inner product): 3.9135535090384366e-13
cond(A_rb) = 15.94084661334877
RB vector norm: 0.43785223630872233
gridfunction 'gfu' on space 'H1HighOrderFESpace(h1ho)'
nested = 1
autoupdate = 1



In [91]:
from ngsolve import *
from netgen.geom2d import unit_square
import numpy as np
import time

start = time.time()

# --- Mesh and function space (Dirichlet on all boundaries for coercivity) ---
mesh = Mesh(unit_square.GenerateMesh(maxh=0.01))
V = H1(mesh, order=1, dirichlet=".*")
u, v = V.TnT()

# --- Mass matrix for M-orthonormalization ---
m = BilinearForm(V)
m += u * v * dx
m.Assemble()
M = m.mat
M_dense = M.ToDense()  # for NumPy-based norms later

# --- Right-hand side (shared across parameters here) ---
f = LinearForm(V)
# f += 100 * v * dx
f += 32 * (y*(1-y)+x*(1-x)) * v * dx
f.Assemble()
b_full = f.vec.FV().NumPy().copy()  # NumPy RHS once and for all

# --- Helper: SPD diffusion tensor ---
def spd_kappa(mu1, mu2, mu3, mu4, eps=1e-8):
    a11, a12, a21, a22 = mu1, mu2, mu3, mu4
    s12 = 0.5 * (a12 + a21)
    return CoefficientFunction(((a11 + eps, s12),
                                (s12, a22 + eps)), dims=(2, 2))

# --- Assemble full-order matrix for a given parameter ---
def assemble_A_full(mu):
    mu1, mu2, mu3, mu4 = mu
    kappa_mu = spd_kappa(mu1, mu2, mu3, mu4)
    a_mu = BilinearForm(V, symmetric=True)
    a_mu += (kappa_mu * Grad(u)) * Grad(v) * dx
    a_mu.Assemble()
    return a_mu.mat  # NGSolve sparse matrix

# --- Snapshot parameters (all SPD by construction) ---
# --- Parameter samples (mu1, mu2, mu3, mu4), all SPD by construction ---
mu_samples = [
    # Isotropic cases
    (1.0, 0.0, 0.0, 1.0),
    (2.0, 0.0, 0.0, 2.0),
    (0.5, 0.0, 0.0, 0.5),

    # Purely diagonal anisotropic
    (1.5, 0.0, 0.0, 0.8),
    (0.8, 0.0, 0.0, 1.4),
    (2.2, 0.0, 0.0, 0.9),

    # Small off-diagonal coupling
    (1.0, 0.3, 0.3, 1.2),
    (0.7, 0.2, 0.2, 1.3),
    (1.5, -0.2, -0.2, 1.8),
    (1.1, 0.4, 0.4, 1.6),

    # Stronger coupling but still SPD
    (2.0, 0.8, 0.8, 2.5),
    (1.8, -0.5, -0.5, 1.4),
    (0.9, 0.6, 0.6, 1.5),

    # Varying magnitudes
    (3.0, 0.0, 0.0, 1.0),
    (1.0, 0.0, 0.0, 3.0),
    (2.5, 0.5, 0.5, 1.2),

    # Mildly ill-conditioned (but SPD-safe with eps if needed)
    (0.6, 0.1, 0.1, 1.5),
    (1.4, 0.2, 0.2, 0.7),
    (1.3, -0.3, -0.3, 0.9),
    (2.1, 0.4, 0.4, 1.9),
]


# --- Generate snapshots by solving full problems ---
snapshots = []
for mu in mu_samples:
    A_mat = assemble_A_full(mu)
    gfu = GridFunction(V)
    # Robust SPD solve via sparse Cholesky
    gfu.vec.data = A_mat.Inverse(V.FreeDofs(), inverse="sparsecholesky") * f.vec
    # Keep a standalone NGSolve vector copy
    vec_copy = gfu.vec.CreateVector()
    vec_copy.data = gfu.vec
    snapshots.append(vec_copy)

# --- M-orthonormal Gram–Schmidt to build reduced basis (RB) ---
def gram_schmidt_M(vectors, M, tol=1e-10):
    RB = []
    for v in vectors:
        w = v.CreateVector()
        w.data = v
        for q in RB:
            Mw = M * w
            alpha = InnerProduct(q, Mw)  # <q, M w>
            w.data -= alpha * q
        Mw = M * w
        nrm = sqrt(InnerProduct(w, Mw))
        if nrm > tol:
            w.data /= nrm
            RB.append(w)
    return RB

RB = gram_schmidt_M(snapshots, M)

print("RB size:", len(RB))

# --- Build explicit W (columns are RB vectors in FE coefficient ordering) ---
W = np.column_stack([rb.FV().NumPy() for rb in RB])  # shape: (N_h, n_RB)

end = time.time()
print(f"Total time: {end - start:.2f} seconds")


RB size: 18
Total time: 12.86 seconds


In [93]:
# --- Projection and lifting using W ---
def project_to_RB(A_full_np, b_full_np, W):
    """
    A_full_np: (N_h, N_h) NumPy array
    b_full_np: (N_h,) NumPy array
    W:         (N_h, n_RB) NumPy array
    """
    A_rb = W.T @ A_full_np @ W
    b_rb = W.T @ b_full_np
    return A_rb, b_rb

def lift_from_RB(u_rb, W):
    return W @ u_rb

# --- Convenience: reduced solve for a given parameter, plus error vs full ---
def solve_reduced_and_compare(mu):
    # Assemble full matrix for mu and convert to NumPy
    A_mat = assemble_A_full(mu)
    A_full_np = A_mat.ToDense()

    # Reduced projection
    A_rb, b_rb = project_to_RB(A_full_np, b_full, W)

    # Reduced solve
    u_rb = np.linalg.solve(A_rb, b_rb)

    # Lift to full space (NumPy vector)
    u_full_rb_np = lift_from_RB(u_rb, W)

    # Reference full FE solve (NGSolve vector -> NumPy)
    gfu_ref = GridFunction(V)
    gfu_ref.vec.data = A_mat.Inverse(V.FreeDofs(), inverse="sparsecholesky") * f.vec
    u_full_ref_np = gfu_ref.vec.FV().NumPy().copy()

    # from ngsolve.webgui import Draw
    # w = Draw(gfu_ref, mesh, "u")
    # w

    # M-norm error and relative M-norm
    e = u_full_rb_np - u_full_ref_np
    num = np.sqrt(e @ (M_dense @ e))
    den = np.sqrt(u_full_ref_np @ (M_dense @ u_full_ref_np))
    rel_err = num / max(den, 1e-16)

    return u_rb, u_full_rb_np, rel_err

start = time.time()

# --- Demo: evaluate at a test parameter (could be new or one of the samples) ---
mu_test = (1, -1, -1, 1.1)
u_rb, u_full_approx, rel_err = solve_reduced_and_compare(mu_test)
end = time.time()
print(f"Total time for test solve: {end - start:.2f} seconds")
print(f"Reduced dimension: {len(u_rb)}")
print(f"Relative M-norm error at mu={mu_test}: {rel_err:.3e}")
# 

Total time for test solve: 6.43 seconds
Reduced dimension: 18
Relative M-norm error at mu=(1, -1, -1, 1.1): 1.916e-02
